In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [70]:
torch.manual_seed(1)

# Multivariable Linear Regression(다중 선형 회귀)

- `Simple Linear Regression(단순 선형 회귀)` : 미지수 $x$가 1개인 선형 회귀
- `Multivariable Linear Regression(다중 선형 회귀)` : 미지수 $x$가 2개 이상인 선형 회귀

## 1. Data Definition

독립 변수 $x$의 개수가 3개인 데이터가 있을때 결과를 예측하는 모델

독립 변수 $x$의 개수가 3개인 일차 방정식을 구하면

$ H(x) = w_1x_1 + w_2x_2 + w_3x_3 + b $

## 2. PyTorch로 구현

In [71]:
# 훈련 데이터 
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [72]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [73]:
# 가설, 비용 함수, 옵티마이저를 선언한 후에 경사 하강법을 사용
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} \
               w1: {w1.item():.3f} \
               w2: {w2.item():.3f} \
               w3: {w3.item():.3f} \
               b: {b.item():.3f} \
               Cost: {cost.item():.6f}')

Epoch    0/1000                w1: 0.294                w2: 0.294                w3: 0.297                b: 0.003                Cost: 29661.800781
Epoch  100/1000                w1: 0.674                w2: 0.661                w3: 0.676                b: 0.008                Cost: 1.563634
Epoch  200/1000                w1: 0.679                w2: 0.655                w3: 0.677                b: 0.008                Cost: 1.497607
Epoch  300/1000                w1: 0.684                w2: 0.649                w3: 0.677                b: 0.008                Cost: 1.435026
Epoch  400/1000                w1: 0.689                w2: 0.643                w3: 0.678                b: 0.008                Cost: 1.375730
Epoch  500/1000                w1: 0.694                w2: 0.638                w3: 0.678                b: 0.009                Cost: 1.319511
Epoch  600/1000                w1: 0.699                w2: 0.633                w3: 0.679                b: 0.009            

## 3. 행렬 연산 사용

입력이나 가중치의 개수가 늘어나면 일일히 선언에 어려움이 있음

행렬 곱셈 연산(또는 벡터의 내적)을 사용해 개선 가능

$ H(X) = w_1x_1 + w_2x_2 + w_3x_3 $

이 식을 행렬(벡터)의 연산으로 표현 하면 아래와 같음

$ \begin{pmatrix} x_1&&x_2&&x_3 \end{pmatrix} \cdot \begin{pmatrix} w_1\\w_2\\w_3 \end{pmatrix} = (x_1w_1 + x_2w_2 + x_3w_3) $

두 벡터를 각각 $X$와 $W$로 표현한다면 다음과 같음

$ H(X) = XW $

- `sample` : 전체 훈련 데이터의 개수를 셀 수 있는 1개의 단위
- `feature` : 각 샘플에서 $y$를 결정하게 하는 각각의 독립 변수 $x$

앞선 훈련 데이터에서 샘플의 수는 총 5개, 특성은 3개

따라서 독립 변수 $x$의 수가 (샘플의 수 * 특성의 수) = 15 개임을 알 수 있음

이를 (샘플의 수 * 특성의 수) 크기의 행렬로 변환 가능하고 이를 $X$ 라 하면 다음과 같음

$ \begin{pmatrix} x_{11}&&x_{12}&&x_{13}\\x_{21}&&x_{22}&&x_{23}\\x_{31}&&x_{32}&&x_{33}\\x_{41}&&x_{42}&&x_{43}\\x_{51}&&x_{52}&&x_{53} \end{pmatrix} $

가중치 벡터 $W$를 곱하면 다음과 같음

$ 
H(X) = XW =
\begin{pmatrix} x_{11}&&x_{12}&&x_{13} \\ x_{21}&&x_{22}&&x_{23} \\ x_{31}&&x_{32}&&x_{33} \\ x_{41}&&x_{42}&&x_{43} \\ x_{51}&&x_{52}&&x_{53} \end{pmatrix} 
\begin{pmatrix} w_1\\w_2\\w_3 \end{pmatrix} =
\begin{pmatrix} 
    x_{11}w_1 + x_{12}w_2 + x_{13}w_3 \\ 
    x_{21}w_1 + x_{22}w_2 + x_{23}w_3 \\ 
    x_{31}w_1 + x_{32}w_2 + x_{33}w_3 \\ 
    x_{41}w_1 + x_{42}w_2 + x_{43}w_3 \\ 
    x_{51}w_1 + x_{52}w_2 + x_{53}w_3 
\end{pmatrix}
$

여기에 각 샘플에 더해지는 편향 $b$를 샘플 수만큼 차원을 가지는 편향 벡터 $B$를 만들어 더하면 다음과 같음

$ 
H(X) = XW + B =
\begin{pmatrix} x_{11}&&x_{12}&&x_{13} \\ x_{21}&&x_{22}&&x_{23} \\ x_{31}&&x_{32}&&x_{33} \\ x_{41}&&x_{42}&&x_{43} \\ x_{51}&&x_{52}&&x_{53} \end{pmatrix} 
\begin{pmatrix} w_1\\w_2\\w_3 \end{pmatrix} +
\begin{pmatrix} b\\b\\b\\b\\b \end{pmatrix} =
\begin{pmatrix} 
    x_{11}w_1 + x_{12}w_2 + x_{13}w_3 + b \\ 
    x_{21}w_1 + x_{22}w_2 + x_{23}w_3 + b \\ 
    x_{31}w_1 + x_{32}w_2 + x_{33}w_3 + b \\ 
    x_{41}w_1 + x_{42}w_2 + x_{43}w_3 + b \\ 
    x_{51}w_1 + x_{52}w_2 + x_{53}w_3 + b
\end{pmatrix}
$

## 4. 행렬 연산을 이용한 PyTorch 구현

In [74]:
#                               x1   x2   x3
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [75]:
# 가중치와 편향 선언
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [76]:
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

In [77]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해집니다.
    hypothesis = x_train.matmul(W) + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} \
            hypothesis: {hypothesis.squeeze().detach()} \
            Cost: {cost.item():.6f}')

Epoch    0/1000             hypothesis: tensor([0., 0., 0., 0., 0.])             Cost: 29661.800781
Epoch  100/1000             hypothesis: tensor([154.0433, 185.0925, 175.8312, 198.5701, 141.2221])             Cost: 5.754573
Epoch  200/1000             hypothesis: tensor([154.0278, 185.0649, 175.9335, 198.5128, 141.2284])             Cost: 5.512386
Epoch  300/1000             hypothesis: tensor([154.0120, 185.0385, 176.0329, 198.4569, 141.2353])             Cost: 5.281667
Epoch  400/1000             hypothesis: tensor([153.9960, 185.0133, 176.1295, 198.4022, 141.2426])             Cost: 5.061868
Epoch  500/1000             hypothesis: tensor([153.9797, 184.9892, 176.2233, 198.3488, 141.2504])             Cost: 4.852424
Epoch  600/1000             hypothesis: tensor([153.9632, 184.9662, 176.3143, 198.2966, 141.2586])             Cost: 4.652705
Epoch  700/1000             hypothesis: tensor([153.9465, 184.9442, 176.4028, 198.2456, 141.2672])             Cost: 4.462287
Epoch  800/1000   